In [4]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import GaussianMixture, KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [5]:
from pyspark.sql.functions import udf, col, struct
from pyspark.sql.types import ArrayType, DoubleType, StringType
from pyspark.mllib.linalg.distributed import RowMatrix, DenseMatrix

In [3]:
cluster_files =  "/Users/simondi/PROJECTS/target_infect_x_project/src/tix-analysis/data/outlier-removal"

In [8]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [7]:
df = spark.read.parquet(cluster_files)

In [1]:
from scipy.spatial import distance

In [ ]:
distance.cdist()

In [148]:
for i in range(2, 10 + 1):
    time1 = time.time()
    km = GaussianMixture(k=i, seed=23)
    model = km.fit(datas)
    time2 = time.time()
    print(i, model.summary.logLikelihood, time2 - time1)

2 -326.46525564568196 0.8249571323394775
3 -311.5342285986531 0.5016491413116455
4 -294.052483271471 0.5052499771118164
5 -263.9393097144451 0.47350287437438965
6 -206.20980517884834 0.36144089698791504
7 -216.66304478449942 0.6501719951629639
8 -103.86915656626311 0.37635302543640137
9 8.433293501608464 0.47348713874816895
10 234.48800703311113 0.624981164932251


In [156]:
model.gaussiansDF.take(2)[1][1]

DenseMatrix(4, 4, [0.2462, 0.0275, -0.193, -0.2179, 0.0275, 0.2935, -0.366, 0.0703, -0.193, -0.366, 0.8844, 0.0509, -0.2179, 0.0703, 0.0509, 0.2532], False)

In [7]:
import scipy
from scipy.stats import multivariate_normal
svd = multivariate_normal.rvs(mean=scipy.ones(5), size=100)

In [172]:
df.take(1)

[Row(label=0, features=DenseVector([0.2978, 1.4635, 0.4609, -0.5065, 1.8492]))]

In [9]:
km = KMeans(seed=5, k=10)
model = km.fit(df)

In [151]:
model.summary.

[Row(label=0, features=DenseVector([1.2011, 0.7218, 0.0537, 2.2096]))]

In [41]:
km = GaussianMixture().setK(5).setSeed(23)
model = km.fit(df)

In [56]:
model.gaussiansDF

DataFrame[mean: vector, cov: matrix]

In [58]:
model.weights

[0.14620840581414205,
 0.15119357104416584,
 0.08541240437382741,
 0.255908435981313,
 0.3612771827865516]

In [125]:
import scipy
import sklearn.mixture
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

In [126]:
scipy.random.seed(23)
X = multivariate_normal.rvs(mean=scipy.ones(10), size=100)

In [127]:
dff = map(lambda x: (int(x[0]), Vectors.dense(x[0:])), X)
df = spark.createDataFrame(dff, schema=["label", "features"])

In [128]:
for i in [100, 90, 80, 70, 60, 50]:
    km = pyspark.ml.clustering.GaussianMixture(k=10, seed=23).fit(df.limit(i))
    sk_gmm = sklearn.mixture.GaussianMixture(10, random_state=23).fit(X[:i, :])
    print(df.limit(i).count(), X[:i, :].shape[0], km.summary.logLikelihood, sk_gmm.score(X[:i, :]))

100 100 368.37475644171036 -1.54949312502
90 90 1026.084529101155 1.16196607062
80 80 2245.427539835042 4.25769131857
70 70 1940.0122633489268 10.0949992881
60 60 2255.002313247103 14.0497823725
50 50 -140.82605873444814 21.2423016046


In [58]:
s = pyspark.ml.clustering.GaussianMixture(k=10, seed=23).fit(df.limit(100))

In [120]:
scipy.linalg.inv(s.gaussiansDF.select("cov").take(10)[10][0].toArray())

IndexError: list index out of range

In [22]:
sk_gmm.score_samples

<bound method BaseMixture.score_samples of GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)>